In [ ]:
from typing import Annotated
from langgraph.graph.message import add_messages
from typing import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END


In [ ]:
"LLM"
llm = init_chat_model(model="openai:gpt-4o-mini")

In [ ]:
"Graph State"

class MyState(TypedDict):
    messages: Annotated[list, add_messages]
    info_complete: bool

In [ ]:
memory = InMemorySaver()
graph_builder = StateGraph(MyState)


In [ ]:
def chatbot(state: MyState):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)


In [ ]:
graph = graph_builder.compile(checkpointer=memory)


In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
config = {"configurable": {"thread_id": "2"}}

In [ ]:
def stream_graph_updates(user_input: str, config):
    events = graph.stream({"messages": [{"role": "user", "content": user_input}]}, config=config, stream_mode="values"):
    for event in events:
        event["messages"][-1].pretty_print()


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input,config)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input,config)
        break